In [4]:
import os
import pandas as pd
import unicodedata

pd.set_option("display.max_rows", 500)


---
### Functions

In [5]:
def unicode_normalize(value: str) -> str:
    """ This function unicode normalizes the provided value which could be a name or address. 
    
    Parameters
    ----------
    value: string
           it is a name or address.


    Returns
    -------
    string
        the returnd value is unicode normalized
    """

    new_value = ""

    for item in value:
        new_item = unicodedata.normalize("NFKD", item)[0]
        new_value = new_value + new_item

    return new_value.casefold()
    
def merge_address(source: pd.DataFrame, edges: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    """This function extracts the address of the entity from source table.
    
    Parameters
    ----------
    source : dataframe
             contains name of the entity
    edges : dataframe
            contains the link between source and address
    address : dataframe
              contains address of the entity

    Returns
    -------
    dataframe
        entity with corresponding address
        
    """

    
    df_address = pd.DataFrame()
    df_address = pd.merge(source, edges, left_on='node_id', right_on='node_id_start')
    df_address = pd.merge(df_address, address, left_on='node_id_end', right_on='node_id', suffixes=('', '_address'), how="inner")
    
    col_source = list(source.columns)
    col_address = ["address", "countries"]

    return df_address[col_source + col_address]

---
### Read Data

In [16]:
absolute_path = os.path.dirname(os.getcwd())
relative_path = "input data"
full_path = os.path.join(absolute_path, relative_path)

In [17]:
# Load data. Data has graph database structure that is nodes and relationships.

officer = pd.read_csv(full_path + "/nodes-officers.csv", low_memory=False, encoding="utf-8")
address = pd.read_csv(full_path + "/nodes-addresses.csv", low_memory=False, encoding="utf-8")
entity = pd.read_csv(full_path + "/nodes-entities.csv", low_memory=False, encoding="utf-8")
intermediary = pd.read_csv(full_path + "/nodes-intermediaries.csv", low_memory=False, encoding="utf-8")
relationships = pd.read_csv(full_path + "/relationships.csv", low_memory=False, encoding="utf-8")



In [18]:
relationships.sample(5)

,node_id_start,node_id_end,rel_type,link,status,start_date,end_date,sourceID
1354799,23000162,20038339,intermediary_of,intermediary of,NaN,NaN,NaN,Bahamas Leaks
1950110,86023362,85019905,officer_of,managing director of,NaN,28-FEB-1996,NaN,Paradise Papers - Aruba corporate registry
1562228,82013094,81027146,registered_address,registered office,NaN,NaN,NaN,Paradise Papers - Appleby
2141679,110049591,100339310,officer_of,director of,NaN,14-APR-2015,NaN,NaN
2422888,240101118,240101125,same_name_as,same name as,NaN,NaN,NaN,NaN


In [9]:
# Not all columns are needed so select only the relevans ones.

col_officer = ["node_id", "name", "sourceID"]
col_address = ["node_id", "address", "countries"]
col_entity = ['node_id', 'name', 'incorporation_date', 'inactivation_date']
col_intermediary = ['node_id', 'name']
col_relationships = ["node_id_start", "node_id_end", "rel_type", "link", "start_date", "end_date"]

officer = officer[col_officer]
address = address[col_address]
entity = entity[col_entity]
intermediary = intermediary[col_intermediary]
relationships = relationships[col_relationships]


In [10]:
# Get the address of the officers.

officer_address = merge_address(officer, relationships, address)
officer_address = officer_address.loc[~officer_address["address"].isna(), :]

officer_address["name"] = officer_address["name"].apply(str).map(unicode_normalize)
officer_address["address"] = officer_address["address"].map(unicode_normalize)

officer_address.sample(5)

,node_id,name,sourceID,address,countries
144577,13012223,emad mohamad taher al-hariri,Panama Papers,no. 702; 7th floor; residence 4 . burj khalifa...,United Arab Emirates
139211,12223841,laurent nordin,Panama Papers,villa 76b; street 26c jumeirah dubai; u.a.e.,United Arab Emirates
381098,56047785,andreas josef dietl,Paradise Papers - Malta corporate registry,dominikanerstr 39 dusseldorf 40545,NaN
38840,12134203,the bearer,Panama Papers,akara building; 24 de castro street; wickhams ...,British Virgin Islands
160523,114806,"lo, tien-an",Offshore Leaks,"pu tuo district jin sha jiang road no. 33, 4th...",China


In [11]:
# Get the address of the entities.

entity_address = merge_address(entity, relationships, address)
entity_address.sample(5)

,node_id,name,incorporation_date,inactivation_date,address,countries
127326,82010517,Cambria Africa PLC,25-OCT-2007,NaN,33-37 Athol Street,Isle of Man
81709,220427,Letterhorne Ltd.,23-APR-2003,NaN,"Kuzniecky & Co. Banco General Building, 21st F...",Panama
248657,55014830,VALE MARITIME LIMITED,17-AUG-1998,NaN,"171 OLD BAKERY STREET, VALLETTA, MALTA",NaN
175313,85038908,MI BUNKER,01-MAY-2008,NaN,NaN,Aruba
153849,85032725,MINA DI ORO LOTTERY,01-JAN-1994,NaN,NaN,Aruba


In [12]:
# Get the address of the intermediaries.

intermediary_address = merge_address(intermediary, relationships, address)
intermediary_address.sample(5)

,node_id,name,address,countries
4106,289338,Carey Administration Limited,Millennium House Ollivier Street Alderney CHAN...,Guernsey
9281,297807,Jeffery T. Ruch,"1800 Santiago Dr Newport Beach, CA 92660 USA",United States
7353,296190,Tai Lee See,2A-2-3 Jalan Batu Uban 5 11700 Pulau Pinang Ma...,Malaysia
2962,291257,"The Goldberg Group, P.C.","4022 North Sheridan Road Chicago, Illinois 606...",United States
8375,298016,Company Express Limited (REFER TO COMMENTS BEF...,"PO Box 14 Clarkes Estate Cades Bay Nevis, WI",Saint Kitts and Nevis


In [13]:
# Get the entities represeted by the intermediaries.

intermediary_entity = pd.merge(intermediary_address, relationships, left_on='node_id', right_on='node_id_start')
intermediary_entity = pd.merge(intermediary_entity, entity_address, left_on='node_id_end', right_on='node_id', suffixes=('', '_entity'))
columns = [
        'name', 
        'address', 
        'countries',
        'rel_type', 
        'link',
        'node_id_entity', 
        'name_entity',
        'address_entity',
        'countries_entity',
        'incorporation_date', 
        'inactivation_date', 
        ]
intermediary_entity[columns].sample(5)

,name,address,countries,rel_type,link,node_id_entity,name_entity,address_entity,countries_entity,incorporation_date,inactivation_date
114437,Portcullis TrustNet (BVI) Limited,"Trust Net Chambers, P.O.Box 3444, Road Town ,...",British Virgin Islands,officer_of,correspondent addr. of,128038,GREAT FUN GROUP INC.,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,26-MAR-1998,NaN
171805,Portcullis TrustNet (BVI) Limited,"32/1 Moo 7, Kwang Annusawalee Khet Bangken Ban...",Thailand,officer_of,records & registers of,137227,E-Consultation Corporation,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,28-JUN-2002,NaN
323063,Portcullis TrustNet (BVI) Limited,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,officer_of,records & registers of,124469,CALIMA TECHNOLOGIES LIMITED,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,29-MAY-1996,NaN
367959,Company Kit Limited,"Unit A, 6/F Shun On Comm Bldg. 112-114 Des Voe...",Hong Kong,intermediary_of,intermediary of,226910,DE SILVER GROUP LIMITED,"Company Kit Limited Unit A, 6/F Shun On Comm B...",Hong Kong,09-JAN-2006,NaN
311442,Portcullis TrustNet (BVI) Limited,c/o Portcullis TrustNet (Singapore) Pte Ltd 6 ...,Singapore,officer_of,records & registers of,157734,Co-Exceed Consulting Limited,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,01-FEB-2008,NaN


In [14]:
# Get the entities and the corresponding entities for officers.

officer_entity_intermediary = pd.merge(officer_address, relationships, left_on='node_id', right_on='node_id_start')
officer_entity_intermediary = pd.merge(officer_entity_intermediary, intermediary_entity, left_on='node_id_end', right_on='node_id_entity', suffixes=('_officer', ''))

officer_entity_intermediary.rename(columns = {"name": "name_intermediary", "address": "address_intermediary"}, inplace=True)
columns = [
        'node_id_officer', 'name_officer', 'address_officer',
        'countries_officer', 'sourceID', 'rel_type_officer',
        'link_officer', 'start_date_officer', 'end_date_officer', 
        'name_entity',
        'address_entity',
        'countries_entity',
        'incorporation_date', 
        'inactivation_date', 
        'name_intermediary', 
        'address_intermediary'
]
officer_entity_intermediary[columns].sample(5)

,node_id_officer,name_officer,address_officer,countries_officer,sourceID,rel_type_officer,link_officer,start_date_officer,end_date_officer,name_entity,address_entity,countries_entity,incorporation_date,inactivation_date,name_intermediary,address_intermediary
2022311,75595,sharecorp limited,jln. mitra sunter boulevard blok a no. 20-21 j...,Indonesia,Offshore Leaks,officer_of,shareholder of,2007-03-08,2009-05-07,Quay Chain Ltd,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,03-JAN-2007,NaN,Portcullis TrustNet (BVI) Limited,"c/o UBS AG, Singapore, One Raffles Quay #50-01..."
9167773,54662,portcullis trustnet (bvi) limited,c/o portcullis trustnet (singapore) pte ltd 6 ...,Singapore,Offshore Leaks,officer_of,records & registers of,NaN,NaN,ANLY TECHNOLOGIES LIMITED,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,05-JUL-2000,NaN,Portcullis TrustNet (BVI) Limited,Jalan Tegehsari Jimbaran Bali Indocnesia
465142,75595,sharecorp limited,"18 link road, jangpura extension, new delhi - ...",India,Offshore Leaks,officer_of,shareholder of,2002-10-16,NaN,S&U Limited,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,30-AUG-2002,NaN,Portcullis TrustNet (BVI) Limited,"32/1 Moo 7, Kwang Annusawalee Khet Bangken Ban..."
170640,75595,sharecorp limited,no. 23 jalan ss22/17 damansara jaya 47400 peta...,Malaysia,Offshore Leaks,officer_of,shareholder of,NaN,NaN,MSRB Consultants Limited,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,28-NOV-1997,NaN,Portcullis TrustNet (BVI) Limited,Jalan Tegehsari Jimbaran Bali Indocnesia
1924276,75595,sharecorp limited,"jl. mangga besar viii/8, jakarta barat, indonesia",Indonesia,Offshore Leaks,officer_of,shareholder of,2007-01-25,NaN,Pearl Energy Worldwide Ltd,Portcullis TrustNet Chambers P.O. Box 3444 Roa...,British Virgin Islands,08-JAN-2007,NaN,PT. Akraya International,Menara Kadin Indonesia Suite 19C JI. H.R. Rasu...


In [22]:
# Write data
relative_path = "output data"
full_path = os.path.join(absolute_path, relative_path)

officer_entity_intermediary[columns].to_csv(full_path + "/officer_entity_intermediary.csv", encoding="utf-8")